# Chapter 1: Crafting a Rust-Based Network Sniffer

[**Network sniffers**](https://en.wikipedia.org/wiki/Packet_analyzer), essential tools in understanding and analyzing data flow across networks, can be crafted from scratch with Rust. Rust's unique blend of performance and safety makes it an ideal choice for low-level network programming. In this exploration, we'll leverage the capabilities of the [**socket2**](https://github.com/rust-lang/socket2) crate, a powerful Rust library that provides abstractions for working with [**raw sockets**](https://en.wikipedia.org/wiki/Network_socket).

Building a network sniffer requires a solid understanding of the fundamentals of raw network packets in Rust. In Rust, handling raw packets involves close interaction with sockets and meticulous byte manipulation. The `socket2` crate simplifies this process, offering abstractions that facilitate efficient packet capture and processing.

Before we dive into the complexities of network sniffing, let's establish a foundation by understanding how Rust manages raw network packets. This understanding is pivotal as we construct our custom network sniffer, allowing us to delve deep into the world of low-level networking.

## 1. Crafting a Rust-Based UDP Host Discovery Tool

In our pursuit to develop a robust [**UDP**](https://en.wikipedia.org/wiki/User_Datagram_Protocol) host discovery tool using Rust, the primary objective is to identify hosts within a target network. This tool holds significance in scenarios where a comprehensive overview of potential targets is sought, facilitating the streamlining of [**reconnaissance**](https://en.wikipedia.org/wiki/Footprinting) and exploitation efforts. The methodology relies on leveraging a well-established behavior exhibited by most operating systems when confronted with UDP datagrams directed at closed ports. Typically, a responsive host generates an ICMP message indicating that the port is unreachable. This [**ICMP**](https://en.wikipedia.org/wiki/Internet_Control_Message_Protocol) response serves as a valuable indicator of an active host, forming the basis for our host discovery mechanism.

The deliberate selection of the User Datagram Protocol (UDP) stems from its inherent advantages in efficiently broadcasting messages across a subnet with minimal overhead. The lightweight nature of this protocol positions it as an ideal candidate for our purposes, emphasizing the goal of maximizing coverage while minimizing potential disruptions. A pivotal aspect of our approach involves the careful selection of a UDP port that is unlikely to be in active use, enabling us to probe multiple ports for comprehensive host coverage.

Understanding the role of UDP in our host discovery tool necessitates a closer examination of its characteristics. UDP, characterized by its connectionless and lightweight nature, aligns seamlessly with our objective of quick and efficient host identification. The absence of overhead associated with connection-oriented protocols renders UDP well-suited for our purpose, as we intend to seamlessly broadcast messages and await ICMP responses. This strategic choice is not arbitrary; rather, it is a deliberate selection based on the efficiency and simplicity afforded by UDP in the context of host discovery.

Delving into the complexities of our approach, the emphasis extends beyond constructing a basic host discovery tool to meticulous decoding and analysis of various network protocol headers. The true potency of our tool lies not only in its ability to identify hosts but also in its capacity to decipher the complex layers of network communications. Decoding network protocol headers necessitates a nuanced understanding of underlying structures, and Rust, with its focus on performance and safety, emerges as a dependable companion in this journey.

As we kick off the implementation of this UDP host discovery tool, the scope extends beyond a singular operating system. The objective is to create a tool transcending platform limitations, catering to both Windows and Linux environments. This versatility is not merely a convenience but a strategic decision to enhance the tool's applicability, particularly in enterprise environments characterized by diverse operating systems.

The prospective evolution of our tool goes beyond basic host discovery. We envision the incorporation of additional logic that could trigger comprehensive [**Nmap**](https://en.wikipedia.org/wiki/Nmap) port scans on any discovered hosts. This strategic enhancement adds a layer of sophistication, allowing for a deeper exploration of the network attack surface associated with each identified host. The decision to leave this as an exercise for the user underscores our commitment to fostering creativity and innovation within the realm of network exploration.

### 1.1 Network Exploration: Decoding the Essence of UDP

Packet sniffing, a fundamental aspect of network analysis, extends on Windows and Linux platforms, each requiring a distinct approach. To ensure adaptability across operating systems, we adopt a strategy that involves creating a socket object and dynamically determining the underlying platform. This platform awareness becomes particularly crucial as the complexities of raw socket access vary between Windows and Linux environments.

On Windows, the process entails additional steps due to the need for setting specific flags via a socket input/output control (IOCTL) mechanism. This mechanism serves as a means of communication between user-space programs and kernel-mode components, facilitating the configuration of network interfaces to operate in [**promiscuous mode**](https://en.wikipedia.org/wiki/Promiscuous_mode). Promiscuous mode, a powerful but privileged state, allows the network interface to capture all incoming packets, regardless of their destination, providing a comprehensive view of network activity. The initiation of promiscuous mode on Windows involves the strategic use of IOCTL to enable the reception of all packets.

In contrast, the Linux counterpart focuses on the specificity of protocols, where the example utilizes the Internet Control Message Protocol (ICMP) for packet sniffing. Linux, by default, requires a more targeted approach, necessitating the selection of a specific protocol for packet capture. The Rust implementation adeptly accommodates these differences, showcasing its platform-aware design.

```rust
use socket2::{Domain, Protocol, Socket, Type};
use std::io::Result;
use std::mem::MaybeUninit;
use std::net::SocketAddr;

fn main() -> Result<()> {
    // Define the host to listen on
    let host: SocketAddr = "0.0.0.0:12345".parse().unwrap();

    let socket_protocol = if cfg!(target_os = "windows") {
        0
    } else {
        1
    };

    // Create a raw socket
    let sniffer = Socket::new(
        Domain::IPV4,
        Type::RAW,
        Some(Protocol::from(socket_protocol)),
    )?;
    // bind to the public interface
    sniffer.bind(&host.into())?;

    // Read one packet
    let mut buffer: [MaybeUninit<u8>; 65535] = unsafe { MaybeUninit::uninit().assume_init() };
    let _ = sniffer.recv_from(&mut buffer)?;
    let raw_buffer: &[u8] =
        unsafe { std::slice::from_raw_parts(buffer.as_ptr() as *const u8, buffer.len()) };

    // Print the captured packet
    println!("{:?}", raw_buffer);

    Ok(())
}
```

The provided Rust code exemplifies the initiation of a raw socket sniffer, starting with the definition of the host IP address to listen on. The subsequent steps involve the creation of a socket object, taking into account the protocol variations between Windows and Linux. In this context, the `cfg!(windows)` macro plays a pivotal role in conditionally determining the platform and adjusting the socket protocol accordingly.

The default configuration of the socket will include IP headers in the captured packets, enhancing the depth of information gathered during the sniffing process. Moreover, the script automatically handles the complexities of promiscuous mode, a critical feature for comprehensive packet capture.

While the provided Rust code captures a single packet for simplicity, it serves as a foundational example for more extensive network analysis tasks. The flexibility of Rust, combined with its platform-aware features, positions it as a reliable choice for crafting network tools that seamlessly operate across diverse operating systems. This illustrative example demystifies the complexities of packet sniffing on Windows and Linux, laying the groundwork for more sophisticated network exploration and analysis endeavors.

In [2]:
:dep socket2 = {version = "0.5.5", features = ["all"]}

In [3]:
use std::process::{Command, Output, Stdio};

// A helper function to execute a shell command from a Rust script
fn execute_command(command: &str) -> Result<(), std::io::Error> {
    let status = Command::new("bash")
        .arg("-c")
        .arg(command)
        .stderr(Stdio::inherit())
        .status()?;

    if status.success() {
        Ok(())
    } else {
        Err(std::io::Error::from_raw_os_error(status.code().unwrap_or(1)))
    }
}

In [11]:
// This following command will execute the sniffer.
// Set your sudo password below by replacing 'your-passowrd' accordingly

let command = "cd decoding-the-essence-of-udp && cargo build && echo 'your-passowrd!' | sudo -S setcap cap_net_raw+ep target/debug/decoding-the-essence-of-udp && target/debug/decoding-the-essence-of-udp";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

// In another terminal or shell window, choose a host to ping, for example: ping google.com

    Finished dev [unoptimized + debuginfo] target(s) in 0.01s


[69, 0, 0, 84, 0, 0, 0, 0, 113, 1, 211, 15, 142, 251, 37, 238, 192, 168, 1, 8, 0, 0, 87, 102, 0, 6, 0, 1, 59, 14, 81, 101, 0, 0, 0, 0, 87, 76, 6, 0, 0, 0, 0, 0, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

()

The displayed output indicates the successful capture of the initial ICMP ping request destined for the specified host. Running this example on Linux would yield the response from the pinged host.

Capturing a single packet is limited in utility, prompting us to enhance the functionality to process more packets and decode their contents. Let's proceed by incorporating additional features into our sniffer code.